In [1]:
import numpy as np
import pandas as pd
%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns
import os
import utility

In [2]:
weather_df = utility.get_processed_dataset()
weather_df.head(4)

,pyranometer,pyroheliometer,azimuth_angle_deg,tower_dry_bulb_temp_deg,tower_wet_bulb_temp_deg,tower_dew_point_deg,tower_rh_pct,peak_wind_speed_mps,avg_wind_direction_deg,station_pressure_mbar,precipitation_mm,snow_depth_cm,moisture,albedometer,datetime_cpy,total_cloud_coverage_pct
datetime,,,,,,,,,,,,,,,,
2020-01-01 00:00:00,-0.962276,0.000000,356.85640,7.216,0.988,-7.312,32.33,9.95,271.3,806.779,0.0,0.219,0.0,0.0,2020-01-01 00:00:00,-1
2020-01-01 00:01:00,-0.937921,0.000000,357.65505,7.251,1.040,-7.260,32.40,8.20,272.9,806.840,0.0,0.206,0.0,0.0,2020-01-01 00:01:00,-1
2020-01-01 00:02:00,-0.944395,0.000000,358.45438,7.256,1.093,-7.207,32.54,6.70,288.8,806.876,0.0,0.148,0.0,0.0,2020-01-01 00:02:00,-1
2020-01-01 00:03:00,-0.951350,-0.029673,359.25416,7.254,1.060,-7.440,31.89,7.70,294.0,806.823,0.0,0.235,0.0,0.0,2020-01-01 00:03:00,-1


### Sampling Data for Training

In [9]:
weather_df_1day = weather_df.loc['2020-01-01 00:00:00' : '2020-01-01 23:59:00'] \
                                .loc[weather_df['total_cloud_coverage_pct'] > -1]
weather_df_1day.head(3)

,pyranometer,pyroheliometer,azimuth_angle_deg,tower_dry_bulb_temp_deg,tower_wet_bulb_temp_deg,tower_dew_point_deg,tower_rh_pct,peak_wind_speed_mps,avg_wind_direction_deg,station_pressure_mbar,precipitation_mm,snow_depth_cm,moisture,albedometer,datetime_cpy,total_cloud_coverage_pct
datetime,,,,,,,,,,,,,,,,
2020-01-01 07:47:00,14.4935,0.079127,123.84491,6.933,1.403,-5.597,38.24,0.7,210.3,802.042,0.0,0.032,0.0,0.1951,2020-01-01 07:47:00,6
2020-01-01 07:48:00,16.5178,-0.034618,124.01096,6.793,1.294,-5.606,38.58,0.7,232.2,801.948,0.0,0.029,0.0,0.1867,2020-01-01 07:48:00,8
2020-01-01 07:49:00,18.6171,-0.153309,124.17734,6.674,1.260,-5.540,39.12,1.7,263.9,801.940,0.0,0.173,0.0,0.1783,2020-01-01 07:49:00,10


In [6]:
training_dataset = utility.series_to_supervised(weather_df_1day)

,pyranometer,pyroheliometer,azimuth_angle_deg,tower_dry_bulb_temp_deg,tower_wet_bulb_temp_deg,tower_dew_point_deg,tower_rh_pct,peak_wind_speed_mps,avg_wind_direction_deg,station_pressure_mbar,precipitation_mm,snow_depth_cm,moisture,albedometer,datetime_cpy,total_cloud_coverage_pct
datetime,,,,,,,,,,,,,,,,
2020-01-01 23:59:00,-0.954779,-0.143420,355.69991,3.523,-1.875,-10.775,30.79,8.70,280.9,800.490,0.0,0.284,0.0,0.0,2020-01-01 23:59:00,-1
2020-01-02 00:00:00,-0.919309,-0.163202,356.49384,3.508,-1.919,-10.919,30.43,11.45,265.8,800.372,0.0,0.318,0.0,0.0,2020-01-02 00:00:00,-1
